# Data Science Bootcamp
# <center> **Aula 14a -- PCA**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D

## 1. Leer datos

In [ ]:
data = pd.read_csv('deport.csv', sep=',', header=0)
data.shape

In [ ]:
data.head()

## 2. Exploración de datos

In [ ]:
# contar si hay datos faltantes
data.isna().sum()

In [ ]:
# tipo de datos
data.dtypes

In [ ]:
# obtener estadísticas simples
data.describe()

In [ ]:
data.boxplot()

In [ ]:
# histogramas

plt.figure()
data.hist(figsize=(6,6))
plt.show()

In [ ]:
# pairplots
plt.figure(figsize=(8,8))
sns.pairplot(data, diag_kind='kde')
plt.show()

## 3. Descomposición SVD

In [ ]:
# convertir los datos a un np.array

names = data.values[:,-1]

X = data.values[:,:-1].astype(np.float32)
X.shape

In [ ]:
type(X)

In [ ]:
X.mean()

In [ ]:
# centramos los datos

mu = X.mean(axis=0)

std = X.std(axis=0)

Xc = (X - mu) / std

In [ ]:
mu

In [ ]:
Xc.mean(axis=0)

In [ ]:
# Otra forma de centrar los datos usando matrices de proyección

#n = X.shape[0]
#one = np.ones(n).reshape(-1,1)

#J = np.eye(n) - one@(one.T)/n       # matriz de proyección
#Xc = J@X                            # centramos los datos
#Xc = Xc / std                       # estandarización

In [ ]:
Xc.mean(axis=0)

In [ ]:
# matriz de covarianzas
Cov = (Xc.T) @ Xc                 # @ = producto matricial

In [ ]:
print(np.round(Cov, 2))

In [ ]:
# descomposición SVD de la matriz Cov(X)
U, S, V = np.linalg.svd(Cov, full_matrices=True)

In [ ]:
print(np.round(U, 2))

In [ ]:
print(np.round(S, 2))

In [ ]:
print(np.round(V.T, 2))

In [ ]:
# Ahora vamos a usar la descomposición SVD de Xc

U, S, V = np.linalg.svd(Xc)

In [ ]:
print(np.round(U, 2))

In [ ]:
U.shape

In [ ]:
print(np.round(V.T, 2))

In [ ]:
V.shape

### 1D

In [ ]:
# proyección a 1D

X1 = Xc @ (V[:1,:].T)   #@np.diag(S[:1])
X1.shape

In [ ]:
plt.figure(figsize=(6,3))
#plt.plot(X1, np.zeros(X1.shape), 'bo')
plt.scatter(X1, np.zeros(X1.shape))
plt.show()

In [ ]:
plt.figure(figsize=(6,3))
plt.hist(X1, bins=17)
plt.show()

### 2D

In [ ]:
# proyección a 2D

X2 = Xc @ (V[:2,:].T)     #@np.diag(S[:2])
X2.shape

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(X2[:,0], X2[:,1])
plt.show()

In [ ]:
fig = plt.figure(figsize=(7,7))
plt.scatter(X2[:,0], X2[:,1])
for i in range(0, X2.shape[0]):
    plt.annotate(names[i], (X2[i,0], X2[i,1]))
plt.show()

### 3D

In [ ]:
# proyección a 2D

X3 = Xc@(V[:3,:].T) #@np.diag(S[:2])
X3.shape

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X3[:,0], X3[:,1], X3[:,2], marker='o')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.show()

## 4. PCA usando scikit-learn

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pcamodel = PCA(n_components=8)
pca2 = pcamodel.fit_transform(Xc)
pca2.shape

In [ ]:
pcamodel.explained_variance_

In [ ]:
pcamodel.explained_variance_ratio_

In [ ]:
plt.bar(range(1,len(pcamodel.explained_variance_ )+1),pcamodel.explained_variance_ )
plt.ylabel('Explained variance')
plt.xlabel('Components')
plt.plot(range(1,len(pcamodel.explained_variance_ )+1),
         np.cumsum(pcamodel.explained_variance_),
         c='red',
         label="Cumulative Explained Variance")
plt.legend(loc='upper left')

In [ ]:
def mybiplot(score, coeff, labels=None):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())
    plt.scatter(xs * scalex,ys * scaley,s=5)
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'r',alpha = 0.5)
        if labels is None:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'green', ha = 'center', va = 'center')
        else:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'g', ha = 'center', va = 'center')
 
    plt.xlabel("PC{}".format(1))
    plt.ylabel("PC{}".format(2))
    plt.grid()

In [ ]:
plt.figure(figsize=(6,6))
mybiplot(pca2[:,0:2], np.transpose(pcamodel.components_[0:2, :]), list(data.columns[:-1]))
plt.show()

In [ ]:
# esta matriz da los componentes principales

components = pcamodel.components_.T
print(np.round(components, 2))